In [3]:
import os
os.environ['HF_DATASETS_OFFLINE']= "1"
os.environ['TRANSFORMERS_OFFLINE']= "1"
import transformer_lens
import datasets
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
from tqdm.auto import tqdm
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.nn.functional as F
from jaxtyping import Float
from functools import partial
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset, Features, Array2D, Array3D
from typing import List, Tuple, Union
import os
import random
import numpy as np
import rouge_score
from rouge_score import rouge_scorer
from time import time
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from copy import deepcopy
import re
datasets.set_caching_enabled(False)
torch.set_grad_enabled(False)

def print_sys_info():
    import psutil
    import socket
    import gpustat
    memory = psutil.virtual_memory()
    print("剩余内存: {} G".format(memory.available / 1024 / 1024 // 1024))
    host_name = socket.gethostname()
    print(f"当前主机名是:{host_name}")
    gpustat.print_gpustat()


def launch_clash():
    import subprocess
    import os

    result = subprocess.run("pidof clash", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if not result.stdout:
        subprocess.Popen("~/tools/clash/clash", shell=True)
        result = subprocess.run("pidof clash", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print(f"Clash is running, pid: {result.stdout}")
    os.environ["http_proxy"] = "http://localhost:7890"
    os.environ["https_proxy"] = "http://localhost:7890"

def close_clash():
    import subprocess
    result = subprocess.run("killall clash", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print(result.stdout)
    !unset http_proxy
    !unset https_proxy

print_sys_info()
# launch_clash()
# close_clash()

剩余内存: 847.0 G
当前主机名是:SH-IDC1-10-140-0-212
SH-IDC1-10-140-0-212      Thu Mar 21 20:12:21 2024  525.60.13
[0] NVIDIA A100-SXM4-80GB | 36°C,  52 % | 76620 / 81920 MB | gaopeng(75708M) hezexin(27M) hezexin(38M) hezexin(45M) hezexin(6M) hezexin(34M) hezexin(18M) hezexin(45M) hezexin(40M) hezexin(43M) hezexin(36M) hezexin(19M) hezexin(128M) hezexin(19M) hezexin(31M) hezexin(6M) hezexin(34M) hezexin(37M) hezexin(37M) hezexin(62M) hezexin(13M) hezexin(54M) hezexin(28M) hezexin(45M) hezexin(35M)
[1] NVIDIA A100-SXM4-80GB | 31°C,   0 % |  5913 / 81920 MB | yangyue(5910M)
[2] NVIDIA A100-SXM4-80GB | 30°C,   0 % |  5913 / 81920 MB | yangyue(5910M)
[3] NVIDIA A100-SXM4-80GB | 30°C,   0 % |  2621 / 81920 MB | yangyue(2618M)
[4] NVIDIA A100-SXM4-80GB | 29°C,   0 % |  5913 / 81920 MB | yangyue(5910M)
[5] NVIDIA A100-SXM4-80GB | 37°C,  53 % | 75711 / 81920 MB | gaopeng(75708M)
[6] NVIDIA A100-SXM4-80GB | 32°C,   0 % |  5913 / 81920 MB | yangyue(5910M)
[7] NVIDIA A100-SXM4-80GB | 46°C,  91 % | 48673 / 8

In [3]:
hf_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
hf_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = HookedTransformer.from_pretrained("gemma-2b", hf_model=hf_model, tokenizer=hf_tokenizer, default_padding_side='left')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b into HookedTransformer


In [7]:
hf_tokenizer.padding_side = "left"
outputs_tokenized = hf_model.generate(hf_tokenizer(["hello world","hello"], padding=True, return_tensors="pt").input_ids, do_sample=True, max_new_tokens=10, num_return_sequences=2, temperature=0.9)
print(outputs_tokenized)
print(hf_tokenizer.batch_decode(outputs_tokenized))

tensor([[     2,  17534,   2134,    575,    498,   1641,    109,    651,   2412,
            603,    476,    585,   1641],
        [     2,  17534,   2134, 235269,   1368,    708,    692,   3900,   3646,
         235336,    109, 235285,   1144],
        [     0,      2,  17534,   1104, 235341, 169692,    109,   1718, 235303,
         235256,  10920,    577,   4675],
        [     0,      2,  17534, 235269,   1368,    798,    590,   5422,    692,
           3646, 235336,    109, 235285]])
['<bos>hello world in c++\n\nThe following is a C++', '<bos>hello world, how are you doing today?\n\nI am', "<pad><bos>hello there! 👋\n\nIt's wonderful to hear", '<pad><bos>hello, how can I assist you today?\n\nI']


In [ ]:
dataset = load_dataset("sst2")['validation']
dataset

In [ ]:
prompt_tmp = "Determine whether the sentiment of this sentence is Positive or Negative.\nSentence:{}\nSentiment:"
tagger = lambda x: " Positive" if x == 1 else " Negative"
test_dst = [(prompt_tmp.format(dataset[i]['sentence']), tagger(dataset[i]['label'])) for i in range(len(dataset))]
for (question, answer) in test_dst[:1]:
    print(question)
    utils.test_prompt(question, answer, model)

In [ ]:
def single_head_ablation(model: HookedTransformer, layer_idx, head_idx, all_cache: Float[torch.Tensor, "l p h d"]):
    model.set_use_attn_result(True)

    def edit_hook(
            result: Float[torch.Tensor, "b p h d"],
            hook: HookPoint,
    ) -> Float[torch.Tensor, "b p h d"]:
        layer_idx = int(hook.name.split('.')[1])
        seq_len = result.shape[1]
        result[:, :, head_idx, :] = einops.repeat(all_cache[layer_idx, :seq_len, head_idx, :], 'p d -> b p d', b=result.shape[0]).to(result.device)
        return result

    model.add_hook(utils.get_act_name('result', layer_idx), edit_hook)
    return model

def single_pattern_ablation(model: HookedTransformer, layer_idx, head_idx, query_idx, key_idx):
    def edit_hook(
            pattern: Float[torch.Tensor, "batch, head_index, query_pos, key_pos"],
            hook: HookPoint,
    ) -> Float[torch.Tensor, "batch, head_index, query_pos, key_pos"]:
        pattern[:, head_idx, query_idx, :] = 0
        pattern[:, head_idx, query_idx, key_idx] = 1
        return pattern

    model.add_hook(utils.get_act_name('pattern', layer_idx), edit_hook)
    return model

def get_random_head_cache(prompt_template='{}', sample_length=200, sample_size=64, seed=42):
    model.set_use_attn_result(True)
    all_tokens = model.to_str_tokens(torch.arange(0, model.tokenizer.vocab_size, dtype=torch.long))
    valid_tokens = [t for t in all_tokens if t.startswith(" ")]
    all_prompt = []
    for i in range(sample_size):
        random.seed(seed + i)
        random_tokens = random.choices(valid_tokens, k=sample_length)
        all_prompt.append(prompt_template.format(''.join(random_tokens)))
    all_cache = {}
    batch_size = 4
    batched_prompt = [all_prompt[i:i + batch_size] for i in range(0, len(all_prompt), batch_size)]
    for batch_prompt in batched_prompt:
        batch_output, batch_cache = model.run_with_cache(batch_prompt, names_filter=lambda x: 'result' in x, device='cpu', padding_side='right')
        for k in batch_cache.keys():
            all_cache[k] = all_cache.get(k, []) + [batch_cache[k]]
    all_cache = dict(sorted(all_cache.items(), key=lambda x: int(x[0].split('.')[1])))
    final_cache = []
    for v in all_cache.values():
        v = [c[:,:sample_length,...] for c in v]
        final_cache.append(torch.vstack(v).mean(0).unsqueeze(0))
    final_cache = torch.cat(final_cache, dim=0)
    # all_cache = torch.cat([torch.vstack(v).mean(0).unsqueeze(0) for v in all_cache.values()], dim=0)  # [layer pos head_idx d_model]
    return final_cache

def full_logit_lens(model: HookedTransformer, prompt: Union[str, List[str]], target_token: Union[str, List[str]], token_idx=-1, k=8):
    model.set_use_attn_result(True)
    prompt = [prompt] if isinstance(prompt, str) else prompt
    prompt_id = model.to_tokens(prompt, padding_side='left')
    target_token = [target_token] if isinstance(target_token, str) else target_token
    target_id = model.to_tokens(target_token, prepend_bos=False)
    logit_lens = torch.zeros((model.cfg.n_heads + 5, model.cfg.n_layers))
    residual_loc = dict(
        hook_result=(0, model.cfg.n_heads),
        hook_resid_pre=(model.cfg.n_heads + 0, model.cfg.n_heads + 1),
        hook_resid_mid=(model.cfg.n_heads + 1, model.cfg.n_heads + 2),
        hook_resid_post=(model.cfg.n_heads + 2, model.cfg.n_heads + 3),
        hook_attn_out=(model.cfg.n_heads + 3, model.cfg.n_heads + 4),
        hook_mlp_out=(model.cfg.n_heads + 4, model.cfg.n_heads + 5),
    )
    annotations = []

    def logit_lens_hook(
            residual: Union[Float[torch.Tensor, "b p d"], Float[torch.Tensor, "b p h d"]],
            hook: HookPoint,
    ):
        layer_idx = int(hook.name.split('.')[1])
        hook_name = hook.name.split('.')[-1]
        loc = residual_loc[hook_name]
        residual_clone = residual.clone()
        if residual_clone.dim() == 3:
            residual_clone = einops.repeat(residual_clone, 'b p d -> b p h d', h=1)
        residual_clone = residual_clone[:, token_idx, :, :]  # [b h d]
        logits = model.unembed(model.ln_final(residual_clone))  # [b h v]
        probs = F.softmax(logits, dim=-1) # [b h v]
        probs = einops.reduce(probs, 'b h v -> h v', 'mean')
        target_probs = torch.zeros_like(logit_lens[loc[0]:loc[1], layer_idx])
        for token_id in target_id:
            target_probs += probs[:, token_id.item()].cpu()  # [h]
        logit_lens[loc[0]:loc[1], layer_idx] = target_probs
        
        topk_probs, topk_indices = torch.topk(probs, k, dim=-1)  # [h k]
        for h_idx in range(topk_indices.shape[0]):
            topk_str_tokens = model.to_str_tokens(topk_indices[h_idx])
            topk_str_tokens = [f"{t[:10]} {topk_probs[h_idx,i].item():.2f}" for i,t in enumerate(topk_str_tokens)]
            annotations.append(
                dict(
                    text='<br>'.join(topk_str_tokens),
                    x=layer_idx,
                    y=h_idx + loc[0],
                    showarrow=False,
                    font=dict(color='white',size=10)
                )
            )
        return residual

    names_filter = lambda x: any([name in x for name in residual_loc.keys()])
    out = model.run_with_hooks(prompt_id, return_type=None, fwd_hooks=[
        (names_filter, logit_lens_hook),
    ])
    fig = go.Figure(data=go.Heatmap(z=logit_lens, colorscale='Viridis_r'))
    x_labels = [f"Layer {i}" for i in range(model.cfg.n_layers)]
    y_labels = [f"Head {i}" for i in range(model.cfg.n_heads)] + ['Resid Pre', 'Resid Mid', 'Resid Post', 'All Heads Out', 'Mlp Out']
    fig.update_layout(
        title=f"{prompt[0]}->[{', '.join(target_token)}]",
        xaxis_title='Layer',
        yaxis_title='Head',
        xaxis=dict(ticktext=x_labels, tickvals=list(range(model.cfg.n_layers))),
        yaxis=dict(ticktext=y_labels, tickvals=list(range(model.cfg.n_heads + 5))),
        height=1600,
        width=1600,
        annotations=annotations,
    )
    fig.show()

def get_head_pattern(model: HookedTransformer, prompt, layer_idx, head_idx):
    out, cache = model.run_with_cache(prompt, names_filter=utils.get_act_name('pattern', layer_idx), device='cpu')
    cache = cache[utils.get_act_name('pattern', layer_idx)]  # [b h pq pk]
    pattern = einops.reduce(cache[:, head_idx, :, :], 'b pq pk -> pq pk', 'mean')
    fig = go.Figure(data=go.Heatmap(z=pattern.T))
    str_tokens = model.to_str_tokens(prompt)
    fig.update_layout(
        title=f"({layer_idx}.{head_idx})", 
        xaxis_title='query', 
        yaxis_title='key',
        xaxis=dict(ticktext=str_tokens, tickvals=list(range(model.cfg.n_layers))),
        yaxis=dict(ticktext=str_tokens, tickvals=list(range(model.cfg.n_heads))),
        height=1000, 
        width=1000
    )
    fig.show()

In [ ]:
random_cache = get_random_head_cache()
zero_cache = torch.zeros_like(random_cache)

In [ ]:
def exp_head_ablation(model: HookedTransformer, test_prompts, test_answers):
    def get_correct_logits_and_probs(model, test_prompts, test_answers):
        out_logits = model(test_prompts)[:, -1, :]
        correct_idxs = model.to_tokens(test_answers, prepend_bos=False)
        out_probs = F.softmax(out_logits, dim=-1)
        correct_logits = torch.diag(out_logits[torch.arange(len(out_logits)), correct_idxs])
        correct_probs = torch.diag(out_probs[torch.arange(len(out_probs)), correct_idxs])
        return correct_logits.mean(), correct_probs.mean()

    model.reset_hooks()
    org_logits, org_probs = get_correct_logits_and_probs(model, test_prompts, test_answers)
    all_lc_matrixs = []
    all_pc_matrixs = []
    cache_list = [zero_cache, random_cache]
    cache_names = ["zero_cache", "random_cache"]
    for cache, name in zip(cache_list, cache_names):
        print(f"running {name}")
        change_in_correct_probs = torch.empty((model.cfg.n_heads, model.cfg.n_layers))
        change_in_correct_logits = torch.empty((model.cfg.n_heads, model.cfg.n_layers))
        for layer_idx in tqdm(range(model.cfg.n_layers)):
            for head_idx in range(model.cfg.n_heads):
                model.reset_hooks()
                model = single_head_ablation(model, layer_idx, head_idx, cache)
                logits, probs = get_correct_logits_and_probs(model, test_prompts, test_answers)
                change_in_correct_probs[head_idx, layer_idx] = probs - org_probs
                change_in_correct_logits[head_idx, layer_idx] = logits - org_logits
        all_lc_matrixs.append(change_in_correct_logits)
        all_pc_matrixs.append(change_in_correct_probs)

    def _plot_heatmap(matrixs, title):
        fig = make_subplots(rows=1, cols=3, subplot_titles=cache_names)
        for i, matrix in enumerate(matrixs):
            fig.add_trace(go.Heatmap(z=matrix, colorscale='Viridis_r', showscale=(i == 0)), row=1, col=i + 1)
        fig.update_layout(title=title, xaxis_title='Layer', yaxis_title='Head', height=400, width=1200)
        fig.show()

    _plot_heatmap(all_pc_matrixs, "Change in correct probs")
    _plot_heatmap(all_lc_matrixs, "Change in correct logits")

In [ ]:
test_prompts = [q for (q,a) in test_dst[:10]]
test_answers = [a for (q,a) in test_dst[:10]]
exp_head_ablation(model, test_prompts, test_answers)

In [ ]:
pos_dataset = [q for (q,a) in test_dst if a == " Positive"]
neg_dataset = [q for (q,a) in test_dst if a == " Negative"]
pos_tokens = [" Positive", "Positive", "positive", " positive"]
neg_tokens = [" Negative", "Negative", "negative", " negative"]
full_logit_lens(model, pos_dataset[:100], pos_tokens, k=5)
full_logit_lens(model, neg_dataset[:100], neg_tokens, k=5)